# Notebook to Practice Folium and Clustering

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
# load LA GeoJSON file
with open('la.json') as json_data:
    la_data = json.load(json_data)

In [79]:
# check file
# la_data

In [4]:
lan = la_data['data']

In [5]:
lan[0][16]

'unincorporated-area'

In [80]:
# # cell to isolate information
# for i in lan:
#     print(i[9])

In [7]:
lan[0][20]

'34.497355239240846'

In [8]:
# define the dataframe columns
column_names = ['neighborhood', 'Type', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,neighborhood,Type,Latitude,Longitude


In [9]:
# create dataframe from json information
neighborhood_names = []
ntype = []
nlat = []
nlong = []

for i in lan:
    
    neighborhood_name = i[9]
    
    neighborhood_type = i[16]
    
    neighborhood_lat = i[20]
    
    neighborhood_lon = i[19]
    
    neighborhood_names.append(neighborhood_name)
    ntype.append(neighborhood_type)
    nlat.append(neighborhood_lat)
    nlong.append(neighborhood_lon)

neighborhoods['neighborhood'] = neighborhood_names
neighborhoods['Type'] = ntype
neighborhoods['Latitude'] = nlat
neighborhoods['Longitude'] = nlong

In [10]:
neighborhoods.shape

(272, 4)

In [11]:
neighborhoods['Type'].value_counts()

segment-of-a-city      114
standalone-city         87
unincorporated-area     71
Name: Type, dtype: int64

In [12]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


In [13]:
# create map of LA using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

In [14]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of this neighborhood are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of this neighborhood are 34.0536909, -118.2427666.


In [15]:
neighborhood_latitude = latitude
neighborhood_longitude = longitude

In [81]:
# CLIENT_ID = '' # your Foursquare ID
# CLIENT_SECRET = '' # your Foursquare Secret
# VERSION = '' # Foursquare API version

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [82]:
# url # display URL

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood Latitude', 
                  'neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [83]:
# la_venues = getNearbyVenues(names=neighborhoods['neighborhood'],
#                                    latitudes=neighborhoods['Latitude'],
#                                    longitudes=neighborhoods['Longitude']
#                                   )

In [52]:
la_venues.head()

,neighborhood,neighborhood Latitude,neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,acton,34.497355239240846,-118.16981019229348,Epik Engineering,34.498612,-118.168249,Construction & Landscaping
1,acton,34.497355239240846,-118.16981019229348,Alma Gardening Co.,34.494762,-118.172550,Construction & Landscaping
2,adams-normandie,34.031461499124156,-118.30020800000011,Orange Door Sushi,34.032270,-118.299541,Sushi Restaurant
3,adams-normandie,34.031461499124156,-118.30020800000011,Shell,34.033095,-118.300025,Gas Station
4,adams-normandie,34.031461499124156,-118.30020800000011,Sushi Delight,34.032445,-118.299525,Sushi Restaurant


In [53]:
la_venues.shape

(2996, 7)

In [84]:
# la_venues.groupby('neighborhood').count()

In [55]:
print('There are {} uniques categories.'.format(len(la_venues['Venue Category'].unique())))

There are 325 uniques categories.


In [56]:
# one hot encoding
la_onehot = pd.get_dummies(la_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['neighborhood'] = la_venues['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,neighborhood,ATM,Accessories Store,Airport,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Notary,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Piano Bar,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Reservoir,Restaurant,River,Road,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sp

In [57]:
la_onehot.shape

(2996, 326)

In [85]:
# la_grouped = la_onehot.groupby('neighborhood').mean().reset_index()
# la_grouped

In [86]:
# num_top_venues = 5

# for hood in la_grouped['neighborhood']:
#     print("----"+hood+"----")
#     temp = la_grouped[la_grouped['neighborhood'] == hood].T.reset_index()
#     temp.columns = ['venue','freq']
#     temp = temp.iloc[1:]
#     temp['freq'] = temp['freq'].astype(float)
#     temp = temp.round({'freq': 2})
#     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#     print('\n')

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
la_venues_sorted = pd.DataFrame(columns=columns)
la_venues_sorted['neighborhood'] = la_grouped['neighborhood']

for ind in np.arange(la_grouped.shape[0]):
    la_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

la_venues_sorted.head()

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,acton,Construction & Landscaping,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
1,adams-normandie,Sushi Restaurant,Taco Place,Park,Gas Station,Playground,Grocery Store,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
2,agoura-hills,Fast Food Restaurant,Breakfast Spot,Sushi Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Gas Station,Lounge,Thai Restaurant,BBQ Joint
3,agua-dulce,Airport,Yoga Studio,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
4,alhambra,Convenience Store,Sporting Goods Shop,Mexican Restaurant,Pizza Place,Breakfast Spot,Hardware Store,Video Store,Fast Food Restaurant,Pet Store,Farmers Market


In [62]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [63]:
# set number of clusters
kclusters = 8

la_grouped_clustering = la_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=0, n_init=12).fit(la_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 4, 5, 4, 1, 4, 4, 4, 4])

In [64]:
# add clustering labels
la_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

la_merged = neighborhoods

# merge l_grouped with la_data to add latitude/longitude for each neighborhood
la_merged = la_merged.join(la_venues_sorted.set_index('neighborhood'), on='neighborhood')
la_merged.head() 

,neighborhood,Type,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,acton,unincorporated-area,34.497355239240846,-118.16981019229348,3.0,Construction & Landscaping,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
1,adams-normandie,segment-of-a-city,34.031461499124156,-118.30020800000011,4.0,Sushi Restaurant,Taco Place,Park,Gas Station,Playground,Grocery Store,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
2,agoura-hills,standalone-city,34.146736499122795,-118.75988450000015,4.0,Fast Food Restaurant,Breakfast Spot,Sushi Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Gas Station,Lounge,Thai Restaurant,BBQ Joint
3,agua-dulce,unincorporated-area,34.504926999796837,-118.3171036690717,5.0,Airport,Yoga Studio,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
4,alhambra,standalone-city,34.085538999123571,-118.13651200000021,4.0,Convenience Store,Sporting Goods Shop,Mexican Restaurant,Pizza Place,Breakfast Spot,Hardware Store,Video Store,Fast Food Restaurant,Pet Store,Farmers Market


In [65]:
la_merged.dropna(axis=0, inplace=True)

In [66]:
la_merged['Cluster Labels'] = la_merged['Cluster Labels'].astype(int)
la_merged.head()

,neighborhood,Type,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,acton,unincorporated-area,34.497355239240846,-118.16981019229348,3,Construction & Landscaping,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
1,adams-normandie,segment-of-a-city,34.031461499124156,-118.30020800000011,4,Sushi Restaurant,Taco Place,Park,Gas Station,Playground,Grocery Store,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
2,agoura-hills,standalone-city,34.146736499122795,-118.75988450000015,4,Fast Food Restaurant,Breakfast Spot,Sushi Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Gas Station,Lounge,Thai Restaurant,BBQ Joint
3,agua-dulce,unincorporated-area,34.504926999796837,-118.3171036690717,5,Airport,Yoga Studio,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
4,alhambra,standalone-city,34.085538999123571,-118.13651200000021,4,Convenience Store,Sporting Goods Shop,Mexican Restaurant,Pizza Place,Breakfast Spot,Hardware Store,Video Store,Fast Food Restaurant,Pet Store,Farmers Market


In [67]:
la_merged.shape

(239, 15)

In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['neighborhood'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [69]:
la_merged.loc[la_merged['Cluster Labels'] == 0, 
                     la_merged.columns[[0,2] + list(range(5, la_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,vermont-slauson,33.983913999124709,Burger Joint,Check Cashing Service,Clothing Store,Sandwich Place,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Film Studio,Ethiopian Restaurant,Electronics Store
31,boyle-heights,34.039168499124116,Grocery Store,Bakery,Donut Shop,Café,Fast Food Restaurant,Japanese Restaurant,Farm,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
47,chesterfield-square,33.983762999124849,Burger Joint,Marijuana Dispensary,Grocery Store,Hardware Store,Flea Market,Fish & Chips Shop,Empanada Restaurant,English Restaurant,Food,Ethiopian Restaurant
53,commerce,33.995079230018334,Coffee Shop,Clothing Store,Yoga Studio,Farm,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant
88,glassell-park,34.114405499123208,Café,Gift Shop,Bar,Pool,Grocery Store,Taco Place,Falafel Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
113,hollywood-hills-west,34.111071499123213,Hotel,Café,Grocery Store,Notary,Italian Restaurant,Film Studio,Falafel Restaurant,Empanada Restaurant,English Restaurant,Flower Shop
116,hyde-park,33.985430999124816,Dance Studio,Indoor Play Area,Caribbean Restaurant,Grocery Store,Convenience Store,Bookstore,Film Studio,Falafel Restaurant,English Restaurant,Food
146,lynwood,33.925213499125533,Grocery Store,Convenience Store,Pharmacy,Park,Yoga Studio,Fabric Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
161,mount-washington,34.103158999123266,Garden,Lounge,Park,Trail,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop
164,north-hills,34.238702499121729,Grocery Store,River,Fast Food Restaurant,Pizza Place,Yoga Studio,Fabric Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant


In [70]:
la_merged.loc[la_merged['Cluster Labels'] == 1, 
                     la_merged.columns[[0,2] + list(range(5, la_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,alondra-park,33.889617004889644,Gym / Fitness Center,Park,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop
18,baldwin-hillscrenshaw,34.01197027055953,Clothing Store,Park,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop
40,chatsworth,34.256403499160569,Park,Food Court,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
96,hacienda-heights,34.002085506418773,Park,Food Court,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
102,harvard-park,33.983762999124835,Park,Bus Stop,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
122,la-crescenta-montrose,34.228709003122127,Park,Coffee Shop,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop
135,la-verne,34.124883999123185,Park,Food Court,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
160,montecito-heights,34.093658999123406,Park,Lake,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
184,porter-ranch,34.278216499201754,Park,Food Court,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
218,south-park,33.996463999124586,Park,Food Court,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm


In [71]:
la_merged.loc[la_merged['Cluster Labels'] == 2, 
                     la_merged.columns[[0,2] + list(range(5, la_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,glendale,34.192975499122355,Trail,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant
158,monrovia,34.148608345800739,Trail,Construction & Landscaping,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant
191,rancho-palos-verdes,33.75869892565828,Trail,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant
211,sierra-madre,34.168112999122641,Trail,Other Repair Shop,Gourmet Shop,Coffee Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
235,tujunga,34.261340090064621,Trail,Lake,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop
241,unincorporated-santa-monica-mountains,34.087645747469352,Trail,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant


In [72]:
la_merged.loc[la_merged['Cluster Labels'] == 3, 
                     la_merged.columns[[0,2] + list(range(5, la_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,acton,34.497355239240846,Construction & Landscaping,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
73,east-pasadena,34.138563505391282,Mexican Restaurant,Construction & Landscaping,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant
104,hasley-canyon,34.476199893867175,Stables,Construction & Landscaping,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
128,lake-los-angeles,34.611038948416571,Construction & Landscaping,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm


In [73]:
la_merged.loc[la_merged['Cluster Labels'] == 4, 
                     la_merged.columns[[0,2] + list(range(5, la_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,adams-normandie,34.031461499124156,Sushi Restaurant,Taco Place,Park,Gas Station,Playground,Grocery Store,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
2,agoura-hills,34.146736499122795,Fast Food Restaurant,Breakfast Spot,Sushi Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Gas Station,Lounge,Thai Restaurant,BBQ Joint
4,alhambra,34.085538999123571,Convenience Store,Sporting Goods Shop,Mexican Restaurant,Pizza Place,Breakfast Spot,Hardware Store,Video Store,Fast Food Restaurant,Pet Store,Farmers Market
6,artesia,33.866895999126271,Indian Restaurant,Chinese Restaurant,Salon / Barbershop,Bubble Tea Shop,Vietnamese Restaurant,Grocery Store,Taiwanese Restaurant,Tea Room,Sandwich Place,Frozen Yogurt Shop
7,altadena,34.193870502232173,Food,Campground,Pharmacy,Notary,Falafel Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop
9,arcadia,34.133229999123017,Bubble Tea Shop,Cosmetics Shop,Gym,Convenience Store,Donut Shop,Chinese Restaurant,Fast Food Restaurant,Food & Drink Shop,Football Stadium,Mexican Restaurant
10,arleta,34.243099999121583,Historic Site,Convenience Store,Video Store,Bakery,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
11,arlington-heights,34.04491049912405,Donut Shop,Karaoke Bar,Liquor Store,Latin American Restaurant,Korean Restaurant,Intersection,Bus Station,Café,Mexican Restaurant,Music Venue
13,atwater-village,34.131066356759177,Taco Place,ATM,Juice Bar,Pizza Place,Pharmacy,Coffee Shop,Other Repair Shop,Electronics Store,Fast Food Restaurant,Mobile Phone Shop
14,avalon,33.336954499133086,American Restaurant,Mexican Restaurant,Playground,Golf Course,Yoga Studio,Farm,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant


In [74]:
la_merged.loc[la_merged['Cluster Labels'] == 5, 
                     la_merged.columns[[0,2] + list(range(5, la_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,agua-dulce,34.504926999796837,Airport,Yoga Studio,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm


In [75]:
la_merged.loc[la_merged['Cluster Labels'] == 6, 
                     la_merged.columns[[0,2] + list(range(5, la_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,beverly-hills,34.082544499123699,Food Truck,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
44,cerritos,33.86689249912628,Food Truck,Korean Restaurant,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant
196,rowland-heights,33.973574999125063,Food Truck,Mobile Phone Shop,Mexican Restaurant,Yoga Studio,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant


In [76]:
la_merged.loc[la_merged['Cluster Labels'] == 7, 
                     la_merged.columns[[0,2] + list(range(5, la_merged.shape[1]))]]

,neighborhood,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
250,vincent,34.09951050296921,Donut Shop,Yoga Studio,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop,Falafel Restaurant,Farm
254,west-covina,34.047182499124105,Donut Shop,Coffee Shop,Japanese Restaurant,Yoga Studio,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Fabric Shop


In [77]:
# la_merged.loc[la_merged['Cluster Labels'] == 8, 
#                      la_merged.columns[[0,2] + list(range(5, la_merged.shape[1]))]]

In [78]:
# la_merged.loc[la_merged['Cluster Labels'] == 9, 
#                      la_merged.columns[[0,2] + list(range(5, la_merged.shape[1]))]]

In [87]:
# save as html file
# map_clusters.save('index.html')